In [1]:
# https://cran.r-project.org/web/packages/penaltyLearning/index.html

In [2]:
# libraries
library(penaltyLearning)
library(data.table)
library(future)
library(future.apply)
library(directlabels)

In [3]:
# Get the names of all folders in a specified directory
datasets <- list.dirs(path = "../../data", full.names = FALSE, recursive = FALSE)

In [4]:
for (dataset in datasets) {
    # read data
    feature.dt <- fread(paste("../../data/", dataset, "/features.csv", sep = ""))
    target.dt  <- fread(paste("../../data/", dataset, "/targets.csv", sep = ""))
    folds.dt   <- fread(paste("../../data/", dataset, "/folds.csv", sep = ""))
    
    # get number of folds
    n_folds <- dim(unique(folds.dt[,"fold"]))[1]

    # combine
    feature.dt.all <- cbind(feature.dt, folds.dt[, "fold"])
    target.dt.all  <- cbind(target.dt, folds.dt[, "fold"])
    
    for (test.fold in 1:n_folds) {
        # get train and test data
        feature.mat.train <- as.matrix(feature.dt.all[feature.dt.all$fold != test.fold, -c("fold")])
        feature.mat.test  <- as.matrix(feature.dt.all[feature.dt.all$fold == test.fold, -c("fold")])
        target.mat.train  <- as.matrix(target.dt.all[target.dt.all$fold != test.fold, -c("fold")])
        
        # train model
        fit <- IntervalRegressionCV(feature.mat = feature.mat.train, target.mat = target.mat.train)
        
        # get prediction from test set
        target.mat.pred <- fit$predict(feature.mat.test)
        colnames(target.mat.pred) <- "pred"

        # save prediction to csv
        prediction <- data.frame(target.mat.pred)
        write.csv(prediction, file = paste("predictions/", paste(dataset, test.fold, "csv", sep = "."), sep=''), row.names = FALSE)
    }
}